In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import wandb

1. Drop Name & Outcome time
2. Convert Age upon intake to single number (in weeks)
3. Split sex upon intake to intact/spayed/neutered and the actual gender (male/female)

- Could build a different model for cats and for dogs

In [2]:
def convert_to_weeks(value):
    value = value.lower()  # Make it case-insensitive
    if 'week' in value:
        # Extract number of weeks
        return int(value.split()[0])
    elif 'month' in value:
        # Convert months to weeks (1 month ≈ 4.345 weeks)
        return int(value.split()[0]) * 4.345
    elif 'year' in value:
        # Convert years to weeks (1 year ≈ 52.1775 weeks)
        return int(value.split()[0]) * 52.1775
    elif 'day' in value:
        # Convert days to 0 weeks
        return 0
    return 0  # In case of unexpected values

def extract_month_year(df, column='Intake Time'):
    """
    Convert the specified datetime column in the DataFrame to two new columns: Month and Year.

    Parameters:
        df (pd.DataFrame): Input DataFrame containing the datetime column.
        column (str): Name of the column to convert (default is 'Intake Time').

    Returns:
        pd.DataFrame: The original DataFrame with additional 'Month' and 'Year' columns.
    """
    # Convert the column to datetime objects. Adjust the format if needed.
    df[column] = pd.to_datetime(df[column], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

    # Extract the month and year from the datetime column
    df['Intake Month'] = df[column].dt.month
    df['Intake Year'] = df[column].dt.year

    return df

In [3]:
train_data = pd.read_csv('train.csv', header=0)
train_data = train_data.drop(columns=['Id', 'Name', 'Outcome Time'])
train_data = train_data.drop(columns=['Found Location', 'Date of Birth'])
train_data = train_data.dropna()
print(train_data.columns)
train_data = pd.get_dummies(train_data, columns=['Intake Condition', 'Intake Type', 'Animal Type', 'Sex upon Intake'])
train_data['Age upon Intake'] = train_data['Age upon Intake'].apply(convert_to_weeks)
train_data = extract_month_year(train_data, column='Intake Time')
train_data = train_data.drop(columns=['Intake Time'])
print(train_data.columns)
# train_data.head()

Index(['Intake Time', 'Intake Type', 'Intake Condition', 'Animal Type',
       'Sex upon Intake', 'Age upon Intake', 'Breed', 'Color', 'Outcome Type'],
      dtype='object')
Index(['Age upon Intake', 'Breed', 'Color', 'Outcome Type',
       'Intake Condition_Aged', 'Intake Condition_Agonal',
       'Intake Condition_Behavior', 'Intake Condition_Congenital',
       'Intake Condition_Feral', 'Intake Condition_Injured',
       'Intake Condition_Med Attn', 'Intake Condition_Med Urgent',
       'Intake Condition_Medical', 'Intake Condition_Neonatal',
       'Intake Condition_Neurologic', 'Intake Condition_Normal',
       'Intake Condition_Nursing', 'Intake Condition_Other',
       'Intake Condition_Parvo', 'Intake Condition_Pregnant',
       'Intake Condition_Sick', 'Intake Condition_Space',
       'Intake Condition_Unknown', 'Intake Type_Abandoned',
       'Intake Type_Euthanasia Request', 'Intake Type_Owner Surrender',
       'Intake Type_Public Assist', 'Intake Type_Stray',
       'Intak

In [4]:
# for col in train_data.columns:
#     uniques = train_data[col].dropna().unique()
#     print(f"Column: {col}")
#     print(uniques[:20])  # limit to first 20 unique values
#     print(f"Total unique (non-null): {len(uniques)}")
#     print("-" * 40)

In [5]:
test_data = pd.read_csv('test.csv', header=0)
test_data.head()

,Id,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Date of Birth
0,1,1/3/19 16:19,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,1/3/17
1,2,10/21/13 7:59,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,9/21/13
2,3,6/29/14 10:38,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,6/29/10
3,4,7/11/15 18:19,Galilee Court And Damita Jo Dr in Manor (TX),Stray,Normal,Dog,Intact Female,5 months,Pit Bull,Brown/White,1/11/15
4,5,2/4/17 10:10,208 Beaver St in Austin (TX),Stray,Injured,Cat,Intact Female,2 years,Domestic Shorthair Mix,Black/White,2/4/15


# Neural Net (MLP)

In [6]:
train_data = train_data.drop(columns=['Breed', 'Color'])

train_x = train_data.drop('Outcome Type', axis=1)
train_y = train_data['Outcome Type']

In [7]:
outcome_mapping = {
	'Return to Owner': 0,
	'Transfer': 1,
	'Adoption': 2,
	'Died': 3,
	'Euthanasia': 4
}

train_y_encoded = train_y.map(outcome_mapping)

X_np = train_x.values.astype(np.float32)
y_np = train_y_encoded.values.astype(np.int64)

X_tensor = torch.tensor(X_np)
y_tensor = torch.tensor(y_np)

# Create a TensorDataset and split 10% for validation
dataset = TensorDataset(X_tensor, y_tensor)
val_size = int(0.1 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [ ]:
# Get the number of input features (columns in train_x) and set hidden layer size and output classes.
input_dim = X_tensor.shape[1]
hidden_dim = 64   # you can adjust this hyperparameter
output_dim = 5    # 5 outcome classes

# Initialize the model.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNet(input_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initialize wandb project
wandb.init(entity='evan-ekuo-edu', project="ML Course Project", config={
    "epochs": 20,
    "batch_size": batch_size,
    "learning_rate": 0.001,
    "hidden_dim": hidden_dim
})
num_epochs = wandb.config.epochs

# ----- Training Loop -----
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # Wrap the dataloader with tqdm for a progress bar.
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for inputs, labels in progress_bar:
        # Move data to GPU if available.
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / train_size
    wandb.log({"epoch": epoch+1, "loss": epoch_loss})
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

In [ ]:
torch.save(model.state_dict(), "model_weights.pth")
print("Model weights saved to model_weights.pth")

In [ ]:
# -------------------------
# Evaluation on the Validation Set
# -------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy*100:.2f}%')
wandb.log({"validation_accuracy": accuracy})